**This file defines the neural networks architecturethat combines two input (Spectrograms and Waveforms)**

Import

In [ ]:
import torch
from torch import nn
from get_feature_extractor import get_feature_extractor # Import from other file in the folder

MultiStream Architecture using two separate feature extraction networks, two ResNet (gave us the best results in the baseline mode), to process two different types of input data simultaneously: spectrograms and waveforms

In [ ]:

class MultiStreamResNet(nn.Module):
    def __init__(self, num_classes, device):
        super().__init__()

        print("Initialization Multi-Stream architecture....")
        self.spectrogram_extractor = get_feature_extractor(device)
        self.waveform_extractor = get_feature_extractor(device)
        #final classification head responsible for combining and interpreting
        #the features from both streams
        self.classifier = nn.Sequential(
            nn.Linear(512 + 512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    #Describes how the input data flows through the two streams and how the
    #resulting features are merged
    def forward(self, spec_data, wave_data):
        spec_features = self.spectrogram_extractor(spec_data)
        wave_features = self.waveform_extractor(wave_data)
        combined_features = torch.cat((spec_features, wave_features), dim=1)
        output = self.classifier(combined_features)
        return output